# Telco Classification Project - Draft Workspace
**Please see this repository for JupyterNB styling (for presentation):**
* https://github.com/faithkane3/example_classification_project

## Overview of work done so far...
- Store contents of telco_churn database to local machine
- acquire.py
- prepare.py

## To-Do
- ~~Write acquire.py and prepare.py~~
- Explore data
- Modeling/Evaluation
- Create polished JupyterNB for presentation
- Solidify all deliverables
- Practice presentation to peer group
- Deliver!

In [1]:
### Imports ###
import pandas as pd
from acquire import gen_telco
from prepare import gen_prep_telco

In [2]:
### Save local copies of the four telco_churn tables to current directory ###
# gen_telco()

In [3]:
### Prepare telco_churn 'customers' Table ###
telco, contract_types, internet_service_types, payment_types = gen_prep_telco()
telco.head(3)

,partner,dependents,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,churn,gender,senior_citizen,internet_service_type_id,contract_type_id,payment_type_id,tenure,monthly_charges,total_charges
0,1,1,1,0,0,1,0,1,1,0,1,0,1,0,1,2,2,9,65.6,593.3
1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,1,2,9,59.9,542.4
2,0,0,1,0,0,0,1,0,0,0,1,1,0,0,2,1,1,4,73.9,280.85
